In [1]:
import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

In [3]:
def train_model(data_yaml_path, epochs=10, batch_size=16, img_size=640, model_size='yolov8s.pt'):
    """Trains a YOLOv8 model.

    Args:
        data_yaml_path: Path to your data.yaml file.
        epochs: Number of training epochs.
        batch_size: Batch size for training.
        img_size: Image size for training.
        model_size: YOLOv8 model size ('yolov8n.pt', 'yolov8s.pt', etc.).

    Returns:
        The path to the saved 'best.pt' model weights.
    """
    model = YOLO(model_size)  # Load the model
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        name='pcb_defects_yolov8'  # A name for the training run
    )
    best_model_path = os.path.join("runs", "detect", results.name, "weights", "best.pt")
    return best_model_path

In [9]:
def process_image(image_path, model, confidence_threshold=0.5):
    """Processes an image to detect PCB defects.

    Args:
        image_path: Path to the PCB image.
        model: A YOLOv8 model instance.
        confidence_threshold: Confidence threshold for defect detection.

    Returns:
        A list of dictionaries, where each dictionary represents a detected defect.
    """
    try:
        if model is None:
            logging.error("Model is not loaded. Cannot process image.")
            return

        if not os.path.exists(image_path):
            logging.error(f"Image not found at: {image_path}")
            return

        results = model.predict(image_path, conf=confidence_threshold)
        
        # Initialize detections as an empty list
        for box in results[0].boxes:
            b = box.xyxy[0].astype(int)
            c = int(box.cls)
            conf = float(box.conf)
            class_name = results[0].names[c]
            detections.append({
                'box': [int(x) for x in b],
                'class': class_name,
                'confidence': conf
            })
        logging.info(f"Defects detected in: {image_path}")
        return detections
    except Exception as e:
        logging.error(f"Error processing image: {e}")
        return

In [11]:
# --- Cell 4: Visualization Function ---
def visualize_detections(image_path, detections):
    """Visualizes the defect detections on the PCB image.

    Args:
        image_path: Path to the PCB image.
        detections: A list of defect detections (as returned by detect_defects).
    """
    img = cv2.imread(image_path)
    for detection in detections:
        box = detection['box']
        class_name = detection['class']
        confidence = detection['confidence']
        label = f"{class_name} {confidence:.2f}"
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(img, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title("Defect Detections")
    plt.axis("off")
    plt.show()

In [ ]:
import os
import cv2
import logging
from ultralytics import YOLO
import matplotlib.pyplot as plt

# --- 1. Configuration ---
DATA_YAML = "PCB-DEFECT-DATASET/data.yaml"  # Path to your data.yaml file
MODEL_SIZE = "yolov8s.pt"  # Model size (n, s, m, l, x)
OUTPUT_DIR = "results"  # Directory to save results

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# --- 2. Data Loading and Preparation (Example - Adapt to your needs) ---
def load_dataset_info(data_yaml_path):
    """Loads information from the data.yaml file."""
    #   (You might add code here to extract class names, etc., if needed)
    return {}  # Placeholder


# --- 3. Model Training ---
def train_model(data_yaml, model_size, epochs=10, batch_size=16):
    """Trains the YOLOv8 model."""
    try:
        model = YOLO(model_size)
        results = model.train(
            data=data_yaml,
            epochs=epochs,
            batch=batch_size,
            name=os.path.basename(model_size).split('.')[0] + "_trained"  # Unique name
        )
        best_model_path = os.path.join("runs", "detect", results.name, "weights", "best.pt")
        logging.info(f"Trained model saved at: {best_model_path}")
        return best_model_path
    except Exception as e:
        logging.error(f"Error during training: {e}")
        return None


# --- 4. Model Evaluation ---
def evaluate_model(model_path, data_yaml):
    """Evaluates the trained model on the validation set."""
    try:
        model = YOLO(model_path)
        metrics = model.val(data=data_yaml)
        metrics_dict = metrics.results_dict  # Access metrics as a dictionary
        logging.info(f"Evaluation metrics: {metrics_dict}")
        return metrics_dict
    except Exception as e:
        logging.error(f"Error during evaluation: {e}")
        return None


# --- 5. Prediction and Visualization ---
def predict_and_visualize(model_path, image_path, output_dir, confidence_threshold=0.5):
    """Performs prediction and visualizes the results."""
    try:
        model = YOLO(model_path)
        results = model.predict(image_path, conf=confidence_threshold)

        img = cv2.imread(image_path)
        for box in results[0].boxes:
            b = box.xyxy[0].astype(int)
            c = int(box.cls)
            conf = float(box.conf)
            class_name = results[0].names[c]
            label = f"{class_name} {conf:.2f}"
            cv2.rectangle(img, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 2)
            cv2.putText(img, label, (b[0], b[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save the visualized image
        os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist
        output_filename = os.path.splitext(os.path.basename(image_path))[0] + "_detected.jpg"
        output_path = os.path.join(output_dir, output_filename)
        cv2.imwrite(output_path, img)
        logging.info(f"Visualization saved to: {output_path}")

        #   (You might return the detections here if needed)
    except Exception as e:
        logging.error(f"Error during prediction/visualization: {e}")


# --- 6. Main Execution ---
if __name__ == "__main__":
    # 6.1 Load dataset information (if needed)
    dataset_info = load_dataset_info(DATA_YAML)

    # 6.2 Train the model
    trained_model_path = train_model(DATA_YAML, MODEL_SIZE, epochs=5)  # Start with fewer epochs

    if trained_model_path:
        # 6.3 Evaluate the model
        evaluation_metrics = evaluate_model(trained_model_path, DATA_YAML)
        print("Evaluation Metrics:", evaluation_metrics)  # Print for quick view

        # 6.4 Predict and visualize (example)
        example_image = "PCB-DEFECT-DATASET/val/images/l_light_01_open_circuit_19_2_600.jpg"  # Example image path
        predict_and_visualize(trained_model_path, example_image, OUTPUT_DIR)
        print(f"Visualization saved in: {OUTPUT_DIR}")
    else:
        logging.error("Model training failed. Skipping evaluation and prediction.")

New https://pypi.org/project/ultralytics/8.3.88 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.85  Python-3.9.7 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=PCB-DEFECT-DATASET/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8s_trained4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

train: Scanning C:\Users\HP\OneDrive\Desktop\PCB project\pcb-defect-dataset\train\labels.cache... 6370 images, 2164 bac
val: Scanning C:\Users\HP\OneDrive\Desktop\PCB project\pcb-defect-dataset\val\labels.cache... 802 images, 264 backgroun


Plotting labels to runs\detect\yolov8s_trained4\labels.jpg... 
"No such keys(s): 'mode.use_inf_as_null'"
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\yolov8s_trained4
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5         0G      3.318      61.87      2.259         19        640:   0%|          | 1/534 [00:56<8:18:59, 5